In [1]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import time
import random

from sklearn.utils import shuffle

from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import cross_val_score

from sklearn.svm import LinearSVC,SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

from sklearn.decomposition import PCA

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import cohen_kappa_score

#from sklearn.datasets import fetch_olivetti_faces
#of= fetch_olivetti_faces()

In [2]:
data=pd.read_table('faces.txt',' ',header=None)

In [3]:
scaler=MinMaxScaler()
scaler.fit(data)
data=scaler.transform(data)
data=pd.DataFrame(data)

In [4]:
#系统抽样
def system_sampling():
    train_list=[]
    test_list=[]
    i=1
    while(len(train_list)!=6):
        random.seed(i)
        tr=random.randint(0,9)
        if tr in train_list:
            pass
        else:
            train_list.append(tr)
        i+=1
    while(len(test_list)!=2):
        random.seed(i)
        te=random.randint(0,9)
        if te in train_list or te in test_list:
            pass
        else:
            test_list.append(te)
        i+=1
    validation_list=[i for i in range(10) if i not in train_list and i not in test_list]

    train_list.sort()
    test_list.sort()
    validation_list.sort()

    train_index=[]
    test_index=[]
    validation_index=[]
    for i in train_list:
        train_index.append(i)
        for j in range(1,40):
            train_index.append(i+j*10)

    for i in test_list:
        test_index.append(i)
        for j in range(1,40):
            test_index.append(i+j*10)

    for i in validation_list:
        validation_index.append(i)
        for j in range(1,40):
            validation_index.append(i+j*10)
    
    return  train_index,validation_index,test_index

In [5]:
pca = PCA(n_components=0.95)#保留95%的方差
pca.fit(data)

PCA(copy=True, iterated_power='auto', n_components=0.95, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [6]:
pca.n_components_

121

In [7]:
data_new = pca.transform(data)
data_new=pd.DataFrame(data_new)
Y = np.array([i for i in range(1,41,1)])
Y = np.repeat(Y, [10]*40, axis=0)
data_new.insert(0,'Y',Y)

train_index,validation_index,test_index=system_sampling()

data_new_test=data_new.loc[test_index]
data_new_train=data_new.loc[train_index]
data_new_validation=data_new.loc[validation_index]

x_train=np.array(data_new_train[data_new_train.columns.difference(['Y'])])
y_train=np.array(data_new_train['Y'])

x_val=np.array(data_new_validation[data_new_validation.columns.difference(['Y'])])
y_val=np.array(data_new_validation['Y'])

x_test=np.array(data_new_test[data_new_test.columns.difference(['Y'])])
y_test=np.array(data_new_test['Y'])

In [8]:
models=[GaussianNB(),SGDClassifier(),KNeighborsClassifier()
        ,LinearSVC(),LogisticRegression(),SVC(kernel='rbf')
        ,DecisionTreeClassifier(),AdaBoostClassifier(),GradientBoostingClassifier()
        ,RandomForestClassifier(),BaggingClassifier(),ExtraTreesClassifier()
        ,XGBClassifier()]

model_names=['GaussianNB','SGDClassifier','KNearestNeighbors'
             ,'LinearSVM','LogisticRegression','rbfSVM'
             ,'DecisionTree','AdaBoostClassifier','GradientBoostingClassifier'
             ,'RandomForestClassifier','BaggingClassifier','ExtraTreesClassifier'
             ,'xgboost'
]

accuracy_new=[]
clf=[]
t_new=[]
i=0
for m in range(len(models)):
    clf.append(models[m])
    time_start = time.time()
    clf[i].fit(x_train,y_train)
    pred=clf[i].predict(x_val)
    accuracy_new.append(accuracy_score(pred,y_val))
    time_end = time.time()
    t_new.append(time_end-time_start)
    i=i+1
result_frame1=pd.DataFrame({'Model':model_names,'Validation set accuracy':accuracy_new,'Time':t_new})

In [9]:
result_frame1

,Model,Validation set accuracy,Time
0,GaussianNB,0.6875,0.005984
1,SGDClassifier,0.7375,0.012965
2,KNearestNeighbors,0.7875,0.006981
3,LinearSVM,0.9375,0.904584
4,LogisticRegression,0.9500,0.200464
5,rbfSVM,0.9250,0.036900
6,DecisionTree,0.4375,0.047874
7,AdaBoostClassifier,0.0250,0.298201
8,GradientBoostingClassifier,0.4500,3.385951
9,RandomForestClassifier,0.4500,0.029920


调参

1.SVM

In [10]:
SVM_time_start1 = time.time()
kernel=['linear','poly','rbf','sigmoid']
for k in kernel:
    clf=SVC(kernel=k)
    clf.fit(x_train,y_train)
    pred=clf.predict(x_val)
    print('Kernel',k,'准确率：',accuracy_score(pred,y_val))
SVM_time_end1 = time.time()

Kernel linear 准确率： 0.9375
Kernel poly 准确率： 0.6125
Kernel rbf 准确率： 0.925
Kernel sigmoid 准确率： 0.775


In [11]:
SVM_time_end1-SVM_time_start1

0.11558866500854492

In [12]:
SVM_time_start2 = time.time()
C=[0.01,0.1,1,10,100,1000]
gamma=[0.01,0.1,1,10,100,1000]
for c in C:
    for g in gamma:
        clf=SVC(C=c,kernel='linear',gamma=g)
        clf.fit(x_train,y_train)
        pred=clf.predict(x_val)
        print('参数：','C:',c,'gamma:',g,'准确率：',accuracy_score(pred,y_val))
SVM_time_end2 = time.time()

参数： C: 0.01 gamma: 0.01 准确率： 0.925
参数： C: 0.01 gamma: 0.1 准确率： 0.925
参数： C: 0.01 gamma: 1 准确率： 0.925
参数： C: 0.01 gamma: 10 准确率： 0.925
参数： C: 0.01 gamma: 100 准确率： 0.925
参数： C: 0.01 gamma: 1000 准确率： 0.925
参数： C: 0.1 gamma: 0.01 准确率： 0.9375
参数： C: 0.1 gamma: 0.1 准确率： 0.9375
参数： C: 0.1 gamma: 1 准确率： 0.9375
参数： C: 0.1 gamma: 10 准确率： 0.9375
参数： C: 0.1 gamma: 100 准确率： 0.9375
参数： C: 0.1 gamma: 1000 准确率： 0.9375
参数： C: 1 gamma: 0.01 准确率： 0.9375
参数： C: 1 gamma: 0.1 准确率： 0.9375
参数： C: 1 gamma: 1 准确率： 0.9375
参数： C: 1 gamma: 10 准确率： 0.9375
参数： C: 1 gamma: 100 准确率： 0.9375
参数： C: 1 gamma: 1000 准确率： 0.9375
参数： C: 10 gamma: 0.01 准确率： 0.9375
参数： C: 10 gamma: 0.1 准确率： 0.9375
参数： C: 10 gamma: 1 准确率： 0.9375
参数： C: 10 gamma: 10 准确率： 0.9375
参数： C: 10 gamma: 100 准确率： 0.9375
参数： C: 10 gamma: 1000 准确率： 0.9375
参数： C: 100 gamma: 0.01 准确率： 0.9375
参数： C: 100 gamma: 0.1 准确率： 0.9375
参数： C: 100 gamma: 1 准确率： 0.9375
参数： C: 100 gamma: 10 准确率： 0.9375
参数： C: 100 gamma: 100 准确率： 0.9375
参数： C: 100 gamma: 1000 准确率： 0.9375
参数：

In [13]:
SVM_time_end2-SVM_time_start2

0.8985977172851562

2.GaussianNB

In [14]:
NB_time_start = time.time()
var_smoothing=[10**-i for i in range(10)]
for v in var_smoothing:
        clf=GaussianNB(var_smoothing=v)
        clf.fit(x_train,y_train)
        pred=clf.predict(x_val)
        print('参数：','var_smoothing:',v,'准确率：',accuracy_score(pred,y_val))
NB_time_end = time.time()

参数： var_smoothing: 1 准确率： 0.725
参数： var_smoothing: 0.1 准确率： 0.8375
参数： var_smoothing: 0.01 准确率： 0.9
参数： var_smoothing: 0.001 准确率： 0.8125
参数： var_smoothing: 0.0001 准确率： 0.725
参数： var_smoothing: 1e-05 准确率： 0.6875
参数： var_smoothing: 1e-06 准确率： 0.6875
参数： var_smoothing: 1e-07 准确率： 0.6875
参数： var_smoothing: 1e-08 准确率： 0.6875
参数： var_smoothing: 1e-09 准确率： 0.6875


In [15]:
NB_time_end-NB_time_start

0.07879066467285156

3.KNearestNeighbors

In [16]:
KNN_time_start = time.time()
n_neighbors=[i for i in range(1,11)]
for n in n_neighbors:
        clf=KNeighborsClassifier(n_neighbors=n)
        clf.fit(x_train,y_train)
        pred=clf.predict(x_val)
        print('参数：','n_neighbors=',n,'准确率：',accuracy_score(pred,y_val))
KNN_time_end = time.time()

参数： n_neighbors= 1 准确率： 0.9125
参数： n_neighbors= 2 准确率： 0.825
参数： n_neighbors= 3 准确率： 0.8125
参数： n_neighbors= 4 准确率： 0.7875
参数： n_neighbors= 5 准确率： 0.7875
参数： n_neighbors= 6 准确率： 0.75
参数： n_neighbors= 7 准确率： 0.7125
参数： n_neighbors= 8 准确率： 0.6375
参数： n_neighbors= 9 准确率： 0.65
参数： n_neighbors= 10 准确率： 0.6125


In [17]:
KNN_time_end-KNN_time_start

0.06383061408996582

4.LogisticRegression

In [18]:
LR_time_start = time.time()
LR_accuracy=[]
penalty=['l1','l2']
C=[0.001, 0.01, 0.1, 1, 10, 100, 1000]
for p in penalty:
    for c in C:
        clf=LogisticRegression(penalty=p,C=c)
        clf.fit(x_train,y_train)
        pred=clf.predict(x_val)
        print('参数：','penalty:',p,'C:',c,'准确率：',accuracy_score(pred,y_val))
        LR_accuracy.append(accuracy_score(pred,y_val))
LR_time_end = time.time()

参数： penalty: l1 C: 0.001 准确率： 0.025
参数： penalty: l1 C: 0.01 准确率： 0.025
参数： penalty: l1 C: 0.1 准确率： 0.55
参数： penalty: l1 C: 1 准确率： 0.9
参数： penalty: l1 C: 10 准确率： 0.925
参数： penalty: l1 C: 100 准确率： 0.925
参数： penalty: l1 C: 1000 准确率： 0.925
参数： penalty: l2 C: 0.001 准确率： 0.7375
参数： penalty: l2 C: 0.01 准确率： 0.9375
参数： penalty: l2 C: 0.1 准确率： 0.9625
参数： penalty: l2 C: 1 准确率： 0.95
参数： penalty: l2 C: 10 准确率： 0.95
参数： penalty: l2 C: 100 准确率： 0.95
参数： penalty: l2 C: 1000 准确率： 0.9375


In [19]:
LR_time_end-LR_time_start

2.078446388244629

5.ExtraTreesClassifier

In [20]:
ET_time_start = time.time()
n_estimators=[100,200,300,400,500,600]
for n in n_estimators:
        clf=ExtraTreesClassifier(n_estimators=n)
        clf.fit(x_train,y_train)
        pred=clf.predict(x_val)
        print('参数：','n_estimators=',n,'准确率：',accuracy_score(pred,y_val))
ET_time_end = time.time()

参数： n_estimators= 100 准确率： 0.875
参数： n_estimators= 200 准确率： 0.8875
参数： n_estimators= 300 准确率： 0.925
参数： n_estimators= 400 准确率： 0.925
参数： n_estimators= 500 准确率： 0.925
参数： n_estimators= 600 准确率： 0.925


In [21]:
ET_time_end-ET_time_start

2.7396795749664307

模型评估

In [22]:
def evaluation(clf):
    clf.fit(x_train,y_train)
    y_pre=clf.predict(x_test)
    print('confusion_matrix:','\n')
    print(confusion_matrix(y_test, y_pre),'\n')
    print('accuracy:',accuracy_score(y_test, y_pre),';','kappa',cohen_kappa_score(y_test, y_pre))

In [23]:
evaluation(SVC(C=1,kernel='linear',gamma=1))

confusion_matrix: 

[[2 0 0 ... 0 0 0]
 [0 2 0 ... 0 0 0]
 [0 0 2 ... 0 0 0]
 ...
 [0 0 0 ... 2 0 0]
 [0 0 0 ... 0 2 0]
 [0 0 0 ... 0 0 2]] 

accuracy: 0.9875 ; kappa 0.9871794871794872


In [24]:
evaluation(GaussianNB(var_smoothing=0.01))

confusion_matrix: 

[[2 0 0 ... 0 0 0]
 [0 2 0 ... 0 0 0]
 [0 0 2 ... 0 0 0]
 ...
 [0 0 0 ... 2 0 0]
 [0 0 0 ... 0 2 0]
 [0 0 0 ... 0 0 2]] 

accuracy: 0.9875 ; kappa 0.9871794871794872


In [25]:
evaluation(KNeighborsClassifier(n_neighbors=1))

confusion_matrix: 

[[0 0 0 ... 1 0 0]
 [0 2 0 ... 0 0 0]
 [0 0 2 ... 0 0 0]
 ...
 [0 0 0 ... 2 0 0]
 [0 0 0 ... 0 2 0]
 [0 0 0 ... 0 0 1]] 

accuracy: 0.9375 ; kappa 0.9358974358974359


In [26]:
evaluation(LogisticRegression(penalty='l2',C=0.1))

confusion_matrix: 

[[2 0 0 ... 0 0 0]
 [0 2 0 ... 0 0 0]
 [0 0 2 ... 0 0 0]
 ...
 [0 0 0 ... 2 0 0]
 [0 0 0 ... 0 2 0]
 [0 0 0 ... 0 0 2]] 

accuracy: 1.0 ; kappa 1.0


In [27]:
evaluation(ExtraTreesClassifier(n_estimators=300))

confusion_matrix: 

[[2 0 0 ... 0 0 0]
 [0 2 0 ... 0 0 0]
 [0 0 2 ... 0 0 0]
 ...
 [0 0 0 ... 2 0 0]
 [0 0 0 ... 0 2 0]
 [0 0 0 ... 0 0 1]] 

accuracy: 0.9875 ; kappa 0.9871794871794872
